In [1]:
#pragma cling add_include_path("/home/sz/Documents/workspace/third_party/minitorch")
#pragma cling add_library_path("/home/sz/Documents/workspace/third_party/minitorch/build/c10")
#pragma cling load("c10")

In [2]:
#pragma cling add_include_path("/home/sz/Documents/workspace/third_party/minitorch/aten/src")
#pragma cling add_library_path("/home/sz/Documents/workspace/third_party/minitorch/build/aten/src/ATen")
#pragma cling load("aten")

In [3]:
#include <vector>
#include <iostream>
#include <optional>

In [4]:
#include <c10/core/TensorImpl.h>

In file included from input_line_10:1:
In file included from /home/sz/Documents/workspace/third_party/minitorch/c10/core/TensorImpl.h:3:
In file included from /home/sz/Documents/workspace/third_party/minitorch/c10/core/Storage.h:3:
In file included from /home/sz/Documents/workspace/third_party/minitorch/c10/core/StorageImpl.h:4:
In file included from /home/sz/Documents/workspace/third_party/minitorch/c10/core/SymInt.h:3:
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:26:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    }
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:28:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:30:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:32:5: e

/home/sz/Documents/workspace/third_party/minitorch/c10/core/MemoryFormat.h:106:5: warning: control may reach end of non-void function [-Wreturn-type]
    }
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/MemoryFormat.h:109:16: note: in instantiation of function template specialization
      'c10::get_channels_last_strides_3d<long>' requested here
        return get_channels_last_strides_3d<int64_t>(sizes);
               ^


In [5]:
std::vector<float> flatted_data_vector1d = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18};
c10:: Device device{c10::DeviceType::CPU};
c10::DataPtr data_ptr{};
data_ptr = c10::DataPtr{flatted_data_vector1d.data(), device};

In [6]:
// step2: create storage
// https://github.com/pytorch/pytorch/blob/master/c10/core/Storage.h
std::size_t size_bytes =72;
c10::Storage storage{/*use_byte_size*/c10::Storage::use_byte_size_t{}, /*size_bytes*/size_bytes, /*data_ptr*/std::move(data_ptr),nullptr, false};

In [7]:
c10::DispatchKey k = c10::DispatchKey::CPU;
c10::DispatchKeySet ks {{k}};
std::cout << ks << std::endl;

input_line_13:3:25: warning: braces around scalar initializer [-Wbraced-scalar-init]
c10::DispatchKeySet ks {{k}};
                        ^~~


DispatchKeySet(CPU)


In [8]:
caffe2::TypeMeta float_dtype = caffe2::TypeMeta::Make<float>();

In [9]:
c10::intrusive_ptr<c10::TensorImpl> impl_ptr = c10::make_intrusive<c10::TensorImpl>(std::move(storage),ks, float_dtype);

In [10]:
c10::IntArrayRef target_size{{3,6}};
impl_ptr->set_sizes_contiguous(target_size);

In [11]:
#include <ATen/core/TensorBase.h>

In [12]:
at::TensorBase tensor_base(impl_ptr);

In [13]:
std::cout << tensor_base.sizes() << std::endl;

[3, 6]


In [14]:
std::cout << tensor_base.strides() << std::endl;

[6, 1]


In [15]:
#include <ATen/templates/TensorBody.h>

In [16]:
at::Tensor tensor_a(impl_ptr);

In [17]:
std::cout << tensor_a.sizes() << std::endl;

[3, 6]


In [18]:
tensor_a.device();

In [19]:
#include <ATen/native/my_kernel.h>

In [20]:
at::my_kernel(tensor_a);

my_kernel_cpu impl in CPU


In [21]:
#include <ATen/native/TensorShape.h>

In [22]:
c10::IntArrayRef new_target_size{{6,3}};
c10::IntArrayRef　new_stride{{3,1}};

input_line_28:3:17: warning: treating Unicode character as whitespace [-Wunicode-whitespace]
c10::IntArrayRef　new_stride{{3,1}};
                ^~


In [24]:
at::Tensor new_a = at::native::as_strided(tensor_a,new_target_size,new_stride, std::nullopt);

In [25]:
std::cout << new_a.sizes() << std::endl;

[6, 3]


In [26]:
std::cout << new_a.strides() << std::endl;

[3, 1]
